In [1]:
import yfinance as yf
import pandas as pd
import datetime as dt
import os

def pull_stock_data(tickers:str, start:str, end:str, interval:str, stripdateindex:str, columns:list) -> pd.DataFrame:
    """
    Returns a dataframe with chosen stock information and treasury bill returns.

    Parameters:
    -------------
    tickers (str): Yahoo finance tickers for companies divided ONLY by single space
    start (str): start date of the period (format yyyy-mm-dd)
    end (str): end date of the period (format yyyy-mm-dd)
    interval (str): wanted interval (1d, 1m, 1y)
    stripdateindex (str): argument for stripping datetime index down (d - day, m - month, y- year)
    columns (list): list of wanted values, args same as in yahoo finance
    """
    # data download
    df = yf.download(tickers = tickers, start = start, end = end, interval = interval, groupby = 'ticker')
    # changing index from datetime to just year and month
    df.index = pd.to_datetime(df.index).to_period(stripdateindex)
    # dropping na rows
    df.dropna(inplace = True)
    return df[columns]
    
def save_to_desktop(dataframe:pd.DataFrame, file_name:str):
    """
    Saves dataframe to desktop in csv format under filename provided

    Parameters:
    ------------
    dataframe (pd.DataFrame): dataframe that is to be saved
    file_name (str): name of the file
    """
    desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
    dataframe.to_csv(desktop+'/'+file_name+'.csv', sep=';', encoding='UTF-8')


In [2]:
tick = "NKE MSFT XOM INTC CAT WMT JPM F UPS MKC ^IRX"
st = '2000-12-01'
en = '2020-12-31'
intv = '1mo'
strpdt = 'm'
cols = ['Adj Close']

dftest = pull_stock_data(tick, st, en, intv, strpdt, cols)
dftest.head(20)

[*********************100%***********************]  11 of 11 completed


C:\Users\maxim\AppData\Local\Temp\ipykernel_18556\3317834269.py:22: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df.index = pd.to_datetime(df.index).to_period(stripdateindex)


Adj Close                                                        \
               CAT          F       INTC        JPM       MKC       MSFT   
Date                                                                       
2000-12  13.426951  12.510591  17.972364  23.754145  5.809333  13.602907   
2001-01  12.549321  15.047398  22.119837  28.748068  5.904675  19.149925   
2001-02  11.895670  15.004970  17.075615  24.571951  6.366356  18.503099   
2001-03  12.690619  15.172237  15.739318  23.645113  6.802116  17.150637   
2001-04  14.354865  15.906024  18.489395  25.267096  6.397518  21.247202   
2001-05  15.596693  13.272593  16.156534  26.096510  6.576582  21.695660   
2001-06  14.413114  13.381606  17.507401  23.611641  6.840294  22.893660   
2001-07  15.867385  13.883081  17.842577  22.990435  6.982386  20.757963   
2001-08  14.493444  10.830654  16.735271  21.076162  7.392922  17.891554   
2001-09  12.986128   9.569784  12.241852  18.267784  7.491059  16.047514   
2001-10  12.962947   8.852737  14.625543  18.915043  7.191118  18.236526   
2001-11  13.848894  10.545339  19.560616  20.375471  7.066224  20.137018   
2001-12  15.259479   8.752522  18.850264  19.635418  6.896962  20.776787   
2002-01  14.684148   8.518672  21.002003  18.393015  7.299296  19.980213   
2002-02  16.330589   8.341923  17.112076  15.949352  8.091984  18.296118   
2002-03  16.724804   9.244511  18.237644  19.439127  8.443731  18.913929   
2002-04  16.068752   8.969813  17.158134  19.139212  8.504745  16.389357   
2002-05  15.474470   9.894822  16.564409  19.793301  8.710402  15.965979   
2002-06  14.491580   9.026226  10.964850  18.675617  8.541233  17.154564   
2002-07  13.233378   7.598950  11.276932  13.742439  7.577253  15.047090   

                                                           
              NKE        UPS        WMT        XOM   ^IRX  
Date                                                       
2000-12  5.512078  33.709507  35.416012  21.869659  5.730  
2001-01  5.440066  35.516903  37.913372  21.168455  4.840  
2001-02  3.859769  32.435711  33.434517  20.388626  4.720  
2001-03  3.990307  32.753685  33.708172  20.482903  4.180  
2001-04  4.141492  33.070271  34.586430  22.404764  3.830  
2001-05  4.071165  34.221516  34.593117  22.442696  3.540  
2001-06  4.159322  33.378876  32.621140  22.197525  3.560  
2001-07  4.716827  32.818703  37.420876  21.229013  3.440  
2001-08  4.959861  31.894709  32.165886  20.410568  3.280  
2001-09  4.643423  30.120930  33.136551  20.141598  2.300  
2001-10  4.902422  29.553057  34.461647  20.167164  2.010  
2001-11  5.262951  32.577904  36.975868  19.119186  1.730  
2001-12  5.585740  31.691910  38.584976  20.208225  1.670  
2002-01  5.956803  33.424789  40.264744  20.079674  1.720  
2002-02  5.852401  34.273769  41.627468  21.236643  1.720  
2002-03  5.966744  35.474045  41.150856  22.671570  1.740  
2002-04  5.307807  35.030605  37.543320  20.778387  1.730  
2002-05  5.349608  35.228981  36.360439  20.654243  1.705  
2002-06  5.339653  36.146214  36.972042  21.288700  1.659  
2002-07  4.911561  38.247665  33.096100  19.124456  1.665

In [3]:
save_to_desktop(dftest, 'data')